In [2]:
# Import necessary libraries
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
import numpy as np
import os

import dagshub


c:\Users\Saiful\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Saiful\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\Saiful\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [4]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

In [5]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})
df.head()

,sentiment,content
1,0,Layin n bed with a headache ughhhh...waitin o...
2,0,Funeral ceremony...gloomy friday...
6,0,"I should be sleep, but im not! thinking about ..."
8,0,@charviray Charlene my love. I miss you
9,0,@kelcouch I'm sorry at least it's Friday?


In [6]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Saiful\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Saiful\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
# difine text preprocessing functions
def lemmatization(text):
    """Lemmatize the text"""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return ' '.join(text)

def remove_stop_words(text):
    """remove stop words from the text"""
    stop_words = set(stopwords.words('english'))
    text = [word for word in text.split() if word not in stop_words]
    return ' '.join(text)

def removing_numbers(text):
    """remove numbers from the text"""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case"""
    text = text.split()
    text = [word.lower() for word in text]
    return ' '.join(text)

def removing_punctuations(text):
    """remove punctuations from the text"""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace(':', '')
    text = re.sub('\s+', ' ', text).strip()
    return text

def removes_urls(text):
    """Remove urls from the text"""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data"""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removes_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f"Error during text normalization: {e}")
        raise

In [8]:
# Normalize the text data
df = normalize_text(df)
df.head()

,sentiment,content
1,0,layin n bed headache ughhhh waitin call
2,0,funeral ceremony gloomy friday
6,0,sleep im not thinking old friend want married ...
8,0,charviray charlene love miss
9,0,kelcouch sorry least friday


In [9]:
mlflow.set_tracking_uri("https://dagshub.com/Saiful0044/Analyzing-Emotional-Sentiment-in-Tweets-Using-Machine-Learning.mlflow")

dagshub.init(repo_owner='Saiful0044', repo_name='Analyzing-Emotional-Sentiment-in-Tweets-Using-Machine-Learning', mlflow=True)

# Set the experiment name
mlflow.set_experiment("Bow vs TfIdf")

Accessing as Saiful0044

Initialized MLflow to track repo "Saiful0044/Analyzing-Emotional-Sentiment-in-Tweets-Using-Machine-Learning"

Repository Saiful0044/Analyzing-Emotional-Sentiment-in-Tweets-Using-Machine-Learning initialized!

<Experiment: artifact_location='mlflow-artifacts:/1ed218a0278b4f0b9a10adda13d04a27', creation_time=1750042595353, experiment_id='1', last_update_time=1750042595353, lifecycle_stage='active', name='Bow vs TfIdf', tags={}>

In [12]:
# Define feature extraction methos
vectorizers = {
    "BoW": CountVectorizer(),
    "IF-IDF": TfidfVectorizer()
}

In [13]:
# define algorithms
algorithms = {
    'LogisticRegression': LogisticRegression(),
    'MultinomialNB': MultinomialNB(),
    'XGBoost': XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier()
}

In [ ]:
# Start the parent run
with mlflow.start_run(run_name="All Experiment") as parent_run:
    # Loop through algorithms and feature extraction methods (Child Runs)
    for algo_name, algorithm in algorithms.items():
        for vec_name, vectorizer in vectorizers.items():
            with mlflow.start_run(run_name=f"{algo_name} with {vec_name}", nested=True) as child_run:
                X = vectorizer.fit_transform(df['content'])
                y = df['sentiment']
                X_train, X_test, y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

                # log preprocessing paraters
                mlflow.log_param('vectorizer', vec_name)
                mlflow.log_param('algorithm', algo_name)
                mlflow.log_param('test_size', 0.2)

                # Model training
                model = algorithm
                model.fit(X_train,y_train)

                # log model parameters
                if algo_name == 'LogisticRegression':
                    mlflow.log_param('C', model.C)
                elif algo_name == 'MultinomialNB':
                    mlflow.log_param('alpha', model.alpha)
                elif algo_name == 'XGBoost':
                    mlflow.log_param('n_estimators', model.n_estimators)
                    mlflow.log_param('learning_rate', model.learning_rate)
                elif algo_name == 'RandomForest':
                    mlflow.log_param('n_estimators', model.n_estimators)
                    mlflow.log_param('max_depth', model.max_depth)
                
                elif algo_name == 'GradientBoosting':
                    mlflow.log_param('n_estimators', model.n_estimators)
                    mlflow.log_param('learning_rate', model.learning_rate)
                    mlflow.log_param('max_depth', model.max_depth)

                # Model evaluation
                y_pred = model.predict(X_test)
                accuracy = accuracy_score(y_test,y_pred)
                precision = precision_score(y_test,y_pred)
                recall = recall_score(y_test,y_pred)
                f1 = f1_score(y_test,y_pred)

                # log evaluation metrics
                mlflow.log_metric('accuracy', accuracy)
                mlflow.log_metric('precision', precision)
                mlflow.log_metric('recall', recall)
                mlflow.log_metric('f1_score', f1)

                # log model
                mlflow.sklearn.log_model(model, 'model')

                notebook_path = 'exp1_bow_vs_tfidf.ipynb'
                os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
                mlflow.log_artifact(notebook_path)
                

                # Print the results for verification
                print(f"Algorithm: {algo_name}, Feature Engineering: {vec_name}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1 Score: {f1}")

2025/06/16 09:32:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
